## <font color='green'> Least Squares Estimation using Matrix Algebra and Numerical Optimization

* Lecture note 2: p.19

In [ ]:
import os
os.chdir('/Users/hj020/Desktop/2022/EconomicAnalytics-master/Python_/Data')

In [ ]:
import numpy as np 
import pandas as pd 
import math 

raw0 = pd.read_csv('College.csv')
raw0['Private']=pd.get_dummies(raw0['Private'],drop_first=True)
raw0.head()

### <font color='green'> 1) Least Squares Estimation using Matrix Algebra

https://web.stanford.edu/~mrosenfe/soc_meth_proj3/matrix_OLS_NYU_notes.pdf

In [ ]:
# convert the dataframe to a numpy array (excluding the first column-college names)
raw00 = raw0.iloc[:,1:].values 

* We can suppress scientific notation using "np.set_printoptions" (e.g. np.set_printoptions(precision=2, suppress=True))
* Scientific notation: https://en.wikipedia.org/wiki/Scientific_notation
* np.set_printoptions: https://numpy.org/doc/stable/reference/generated/numpy.set_printoptions.html

In [ ]:
raw00

In [ ]:
np.set_printoptions(precision=2, suppress=True)

In [ ]:
raw00

In [ ]:
# Construct X matrix
X=raw00[:,(4,0,8,11,16)] # select predictors (note that the first column was removed)
nrow = X.shape[0]
intcpt = np.ones( (nrow,1), ) # create an intercept
X = np.concatenate((intcpt, X), axis=1) # add the intercept to X (i.e X = [intcpt,X] )

In [ ]:
# Construct Y vector
Y=raw00[:,15]

* The code above didn't specify whether it is a row or column vector
* Use Y=raw00[:,[15]] to specify it is a column vector

####  <font color='green'> i) Compute LS estimates

$\hat{\beta} = (X^{'}X)^{-1}X^{'}Y$
    
* inv( ) from numpy.linalg
* transpose function
* matrix multiplication

In [ ]:
from numpy.linalg import inv
OLSres = inv(X.T@X)@(X.T@Y)
print(OLSres) # Compare this to the previous result obtained from the statsmodels package

####  <font color='green'> ii) Compute the Standard Errors of the Estimates & t-statistics

$\hat{\sigma}_{\beta} = Diag \left(\sqrt{\hat{\sigma}^2(X^{'}X)^{-1}} \right)$ where $\hat{\sigma}^2 = \hat{U}^{'}\hat{U}/(n-p-1)$ and $\hat{U} = Y - X\hat{\beta}$
    
$t_{\beta} = | \hat{\beta}/\hat{\sigma}_{\beta} |$
    

In [ ]:
# Calculate Residuals
resid = Y-X@OLSres
# Calculate SER (Standard error of the regression)
SER = (resid.T@resid)/(nrow-X.shape[1])
# Calculate SE
SE = np.sqrt(np.diag(SER*inv(X.T@X))) # Compare this to the previous result from the statsmodels package

In [ ]:
# Calculate T statistics
Tstat = abs(OLSres/SE)

In [ ]:
Tstat

### <font color='green'> 2) Least Squares Estimation using Numerical Optimization

$\hat{\beta} = argmin_{\beta} (Y - X\beta)^{'}(Y - X\beta)/n$
    
* Newton Method : https://en.wikipedia.org/wiki/Newton%27s_method_in_optimization
* Optimization in Python: https://scipy-lectures.org/advanced/mathematical_optimization/ & https://realpython.com/python-scipy-cluster-optimize/

In [ ]:
from scipy import optimize

In [ ]:
# Define loss fn in two ways

# loss function 1
def loss(inpt,Y,X):
    nrow=Y.shape[0]
    loss0=0

    for i in range(0,nrow):
        
            resid = Y[i]-X[i,:]@inpt
            loss0 = loss0+resid*resid
            # can be done simply: loss0+=resid*resid (add and assign)
            
    return loss0

# loss function 2
def loss2(inpt,Y,X):

    resid = Y-X@inpt
    loss0 = resid.T@resid
            
    return loss0

* Optimizer "fmin" : https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.fmin.html

In [ ]:
# Optimize
inpt = np.zeros((X.shape[1],1)) # starting value
OLSres2=optimize.fmin(loss2,
                      inpt,
                      args=(Y,X),
                      maxfun=40000,
                      maxiter=40000,
                      ftol=1e-10,
                      xtol=1e-10,
                      disp=True
                     )

In [ ]:
print(OLSres2)

### <font color='darkred'> HW3
* Pick one of your linear regression models in HW2 
* Compute least squares estimates, standard errors of the estimates and t-statistics <ins> using the matrix algebra and optimization algorithm as described above </ins>
* Compare them to the results previously obtained from the statsmodels package